In [1]:
import os
import random
import yaml
import torch
import torch.cuda as cuda
from datetime import datetime
import mlflow
from box import Box
import cv2
import matplotlib.pyplot as plt
import pytorch_lightning as pl

from src.datamodule import CCDataModule
from src.encoder import CLIPEncoder, ResNetEncoder
from src.classifier import CCClassifierSmall, CCClassifierLarge
from src.module import CCModule
from src.callbacks import get_callbacks
from src.logger import get_logger
from src.trainer import get_trainer

os.chdir('../')
torch.set_float32_matmul_precision('medium')

/home/gptrapletti/ds/.miniconda/envs/clipclass/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test and compute metrics

In [7]:
with open('config.yaml', 'r') as f:
    cfg = Box(yaml.safe_load(f))

In [8]:
datamodule = CCDataModule(
    encoder_name=cfg.encoder_name,
    image_path=cfg.image_path,
    gt_path=cfg.gt_path,
    val_size=cfg.val_size,
    test_size=cfg.test_size,
    batch_size=cfg.batch_size,
    seed=cfg.seed,
)

datamodule.prepare_data()
datamodule.setup('train')
dl = datamodule.val_dataloader()

In [ ]:
ckp_filepath = 'checkpoints/20231221-161459/epoch=4.ckpt'
# model = CCModule.load_from_checkpoint(ckp_filepath)
model = CCModule.load_from_checkpoint(ckp_filepath, map_location=torch.device('cpu'))

/opt/conda/envs/cc3/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.1.2, which is newer than your current Lightning version: v1.9.5
  rank_zero_warn(
/opt/conda/envs/cc3/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
/opt/conda/envs/cc3/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(


In [ ]:
trainer.test(model, dataloaders=dl)

In [ ]:
model.eval()

preds_ls = []
for batch in dl:
    images, labels = batch
    print(images.shape, labels.shape)
    preds = model(images)
    preds_ls.append(preds)

print(images.norm().item(), preds.norm().item())

In [ ]:
preds_ls[0].shape

In [ ]:
probs = torch.nn.functional.softmax(preds_ls[0], dim=1)
probs = torch.round(probs, decimals=2)
probs.shape

In [ ]:
pred_labels = torch.argmax(probs, dim=1)
pred_labels

In [ ]:
gt_labels = torch.argmax(labels, dim=1)
gt_labels

In [ ]:
accuracy = (pred_labels == gt_labels).sum() / pred_labels.shape[0]
accuracy

In [ ]:
from torchmetrics import Accuracy
accuracy_fn = Accuracy(task="multiclass", num_classes=3)
accuracy_fn(preds=pred_labels, target=gt_labels)

## Create PDFs

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm

correct_predictions_pdf = PdfPages('output/correct_predictions.pdf')
wrong_predictions_pdf = PdfPages('output/wrong_predictions.pdf')

for path, pred, gt in tqdm(zip(datamodule.val_filepaths, preds, gts), total=len(preds)):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.title(os.path.basename(path))
    plt.axis('off')
    text = f'GT: {mapping[gt]} - PRED: {mapping[pred]}'
    plt.figtext(0.5, 0.05, text, ha="center", fontsize=12)

    if pred == gt:
        correct_predictions_pdf.savefig()
    else:
        wrong_predictions_pdf.savefig()

    plt.close()

correct_predictions_pdf.close()
wrong_predictions_pdf.close()

## Predict

In [25]:
with open('config.yaml', 'r') as f:
    cfg = Box(yaml.safe_load(f))

In [26]:
datamodule = CCDataModule(
    encoder_name=cfg.encoder_name,
    image_path=cfg.image_path,
    gt_path=cfg.gt_path,
    val_size=cfg.val_size,
    test_size=cfg.test_size,
    batch_size=cfg.batch_size,
    seed=cfg.seed,
)

datamodule.prepare_data()
datamodule.setup('train')
dl = datamodule.val_dataloader()

In [27]:
model = CCModule.load_from_checkpoint(cfg.best_ckp_path)

## Model gets loaded in the GPU because it's in its internal configs/hyperparams.
## To load it directly into the CPU use this instead
# model = CCModule.load_from_checkpoint(ckp_filepath, map_location=torch.device('cpu'))

/home/gptrapletti/ds/.miniconda/envs/clipclass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
/home/gptrapletti/ds/.miniconda/envs/clipclass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(


In [28]:
trainer = pl.Trainer(
    max_epochs=cfg.epochs,
    logger=None,
    callbacks=None,
    log_every_n_steps=5,
    gradient_clip_val=0.5,
    accelerator='gpu',
    devices=[0],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
model.eval()
pred = trainer.predict(dataloaders=dl, model=model)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  5.89it/s]


In [ ]:
# V2
# model.to('cuda:0')

datamodule.prepare_data()
datamodule.setup('train')
dl = datamodule.val_dataloader()

model.eval()

preds_ls = []
for batch in dl:
    images, labels = batch
    images = images.to('cuda:0')
    labels = labels.to('cuda:0')
    preds = model(images)
    preds_ls.append(preds)
    
# preds_ls = []
# for i in range(100):
#     for batch in dl:
#         images, labels = batch
#         images = images.to('cuda:0')
#         labels = labels.to('cuda:0')
#         preds = model(images)
#         preds_ls.append(preds)
    
len(preds_ls)

## Evaluate

In [14]:
with open('config.yaml', 'r') as f:
    cfg = Box(yaml.safe_load(f))

In [15]:
datamodule = CCDataModule(
    encoder_name=cfg.encoder_name,
    image_path=cfg.image_path,
    gt_path=cfg.gt_path,
    val_size=cfg.val_size,
    test_size=cfg.test_size,
    batch_size=cfg.batch_size,
    seed=cfg.seed,
)

datamodule.prepare_data()

In [4]:
ckp_filepath = 'checkpoints/20231221-231007/epoch=2.ckpt'
model = CCModule.load_from_checkpoint(ckp_filepath)

/opt/conda/envs/cc3/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
/opt/conda/envs/cc3/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(


In [ ]:
model.eval()
model.to('cuda:0')
model.evaluate(datamodule, 'cuda:0')

## Predict via callback

In [2]:
with open('config.yaml', 'r') as f:
    cfg = Box(yaml.safe_load(f))

In [3]:
datamodule = CCDataModule(
    encoder_name=cfg.encoder_name,
    image_path=cfg.image_path,
    gt_path=cfg.gt_path,
    val_size=cfg.val_size,
    test_size=cfg.test_size,
    batch_size=cfg.batch_size,
    seed=cfg.seed,
)

In [4]:
model = CCModule.load_from_checkpoint(cfg.best_ckp_path)

/home/gptrapletti/ds/.miniconda/envs/clipclass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
/home/gptrapletti/ds/.miniconda/envs/clipclass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.
  rank_zero_warn(


In [5]:
callbacks = get_callbacks(ckp_path=cfg.ckp_path)

In [6]:
trainer = pl.Trainer(
    max_epochs=cfg.epochs,
    logger=None,
    callbacks=callbacks,
    log_every_n_steps=5,
    gradient_clip_val=0.5,
    accelerator='gpu',
    devices=[0],
    enable_progress_bar=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
model.eval()
pred = trainer.predict(model=model, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Report generated!


In [11]:
pred[0].shape

torch.Size([16, 3])